# 😜 Joke Generator

It generate jokes and separate the setup from punchline.

In [6]:
from os import environ

if not ("OPENAI_API_KEY" in environ):
    raise Exception("OPENAI API KEY is required")

### Setup **`OPENAI_MODEL`** Model

In [7]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### Create Structured Output Schema

In [8]:
from pydantic import BaseModel, Field
from typing import Optional


class JokeSchema(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke.")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[str] = Field(
        description="How funny the joke is, from 1 to 10")

### Using `TypedDict` Class

- Instead of using pydantic we can also use `TypedDict` class to create structured output.
- We can optionally use a special Annotated syntax supported by LangChain that allows you to specify the default value and description of a field.

In [9]:
from typing import Optional
from typing_extensions import Annotated, TypedDict


class Joke(TypedDict):

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None,
                      "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Project `proj_DYnvYlDwHkLhf8itjcuCpsTX` does not have access to model `gpt-4o-mini`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

### We can also use `JSON` to make Schema.

In [13]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about cats")

{'setup': 'Why was the cat sitting on the computer?',
 'punchline': 'Because it wanted to keep an eye on the mouse!',
 'rating': 7}

### Initialize Model is `Structured Output`

In [6]:
llm_with_structured_output = llm.with_structured_output(JokeSchema)

### Run the model

In [8]:
llm_with_structured_output.invoke("Tell me a joke.")

JokeSchema(setup='Why did the scarecrow win an award?', punchline='Because he was outstanding in his field!', rating='8')

In [10]:
llm_with_structured_output.invoke("How are you today?")

JokeSchema(setup='How do you organize a space party?', punchline='You planet!', rating='7')

### Choosing between multiple schema.

- The simples way to let the model choose from multiple schemas is to create a parent schema that has union-typed attribute.

In [10]:
from pydantic import BaseModel, Field
from typing import Optional, Union


class Joke(BaseModel):
    """Joke to tell user"""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")
    rating: Optional[int] = Field(
        description="How funny the joke is, from 1 to 10")


class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(
        description="A conversational response to the user's query")


class FinalResponse(BaseModel):
    final_output: Union[Joke, ConversationalResponse]


structured_llm = llm.with_structured_output(FinalResponse)

structured_llm.invoke("Tell me a joke about cats")

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Project `proj_DYnvYlDwHkLhf8itjcuCpsTX` does not have access to model `gpt-4o-mini`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [16]:
structured_llm.invoke("How are you today?")

FinalResponse(final_output=ConversationalResponse(response="I'm just a program, but I'm here and ready to help you! How about you? How's your day going?"))

- Same thing we can achieve using `TypedDict`  also

## Steaming

- We can stream the outputs from the our structured model when the output type is dict (i.e when the schema is specified as a TypedDict class or JSON Schema dict).)

In [4]:
from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None,
                      "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)


for chunk in structured_llm.stream("Tell me a joke about cats"):
    print(chunk)

NameError: name 'Optional' is not defined

## Using few shot prompting

- For more complex schema it's useful to use few-shot examples to the prompt.
- This can be done, with using simplest and most common way via system message in a prompt 

In [32]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""


prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=system),
    ("human", "{input}")
])


few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("what's something funny about woodpeckers")

{'setup': 'Woodpecker',
 'punchline': "Woodpecker knock knock, who's there? Just me, I'm just pecking around!",
 'rating': 6}

- When the underlying method for structuring outputs is tool calling, we can pass in our examples as explicit tool calls.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage



examples = [



    HumanMessage("Tell me a joke about planes", name="example_user"),



    AIMessage(
        "",



        name="example_assistant",



        tool_calls=[



            {



                "name": "joke",



                "args": {



                    "setup": "Why don't planes ever get tired?",



                    "punchline": "Because they have rest wings!",



                    "rating": 2,



                },



                "id": "1",



            }



        ],



    ),



    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.



    ToolMessage("", tool_call_id="1"),



    # Some models also expect an AIMessage to follow any ToolMessages,



    # so you may need to add an AIMessage here.



    HumanMessage("Tell me another joke about planes", name="example_user"),



    AIMessage(
        "",



        name="example_assistant",



        tool_calls=[



            {



                "name": "joke",



                "args": {



                    "setup": "Cargo",



                    "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",



                    "rating": 10,



                },



                "id": "2",



            }



        ],



    ),



    ToolMessage("", tool_call_id="2"),



    HumanMessage("Now about caterpillars", name="example_user"),



    AIMessage(
        "",



        tool_calls=[



            {



                "name": "joke",



                "args": {



                    "setup": "Caterpillar",



                    "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",



                    "rating": 5,



                },



                "id": "3",



            }



        ],



    ),



    ToolMessage("", tool_call_id="3"),



]



system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \



Return a joke which has the setup (the response to "Who's there?") \



and the final punchline (the response to "<setup> who?")."""



prompt = ChatPromptTemplate.from_messages(



    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]



)



few_shot_structured_llm = prompt | structured_llm



few_shot_structured_llm.invoke({"input": "crocodiles", "examples": examples})

NameError: name 'structured_llm' is not defined

## Raw Output

- LLM aren't perfect to generate structured output, especially when schema is too complex. You can avoid raising exceptions and handling the raw output by passing `include_raw=True`. 
- This changes the output format to contain the raw message output, the parsed value (if successful), and any resulting errors:

In [34]:
structured_llm = llm.with_structured_output(Joke, include_raw=True)

structured_llm.invoke("Tell me a joke about cats")

{'raw': AIMessage(content='{"setup":"Why was the cat sitting on the computer?","punchline":"Because it wanted to keep an eye on the mouse!","rating":7}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 96, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'stop', 'logprobs': None}, id='run-d63e5d0e-b9ef-4d29-9694-81cac1845963-0', usage_metadata={'input_tokens': 96, 'output_tokens': 34, 'total_tokens': 130, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'parsed': {'setup': 'Why was the cat sitting on the computer?',
  'punchline': 'Because it wanted to keep an eye on the mous